1. 종가만 가져와서 데이터 프레임
2. 데이터를 옆으로 보내기

In [16]:
import pymysql
import pandas as pd

def db_connect():
    db = pymysql.connect(host="localhost", port=3306,
                    user="root",passwd="pass",
                    db="marketdb")
    return db

db=db_connect()
cursor=db.cursor()

cursor.execute("select stock_code, stock_date, close_price from ohlcv where stock_code = 005930")
samsung = cursor.fetchall()

df=pd.DataFrame(samsung, columns=["stock_code","stock_date","close_price"])
df

,stock_code,stock_date,close_price
0,005930,2020-01-02,55200
1,005930,2020-01-03,55500
2,005930,2020-01-06,55500
3,005930,2020-01-07,55800
4,005930,2020-01-08,56800
5,005930,2020-01-09,58600
6,005930,2020-01-10,59500
7,005930,2020-01-13,60000
8,005930,2020-01-14,60000
9,005930,2020-01-15,59000


### sql로 한 번에 가져오기

In [13]:
db=db_connect()
cursor = db.cursor()

sql = """
    SELECT o.stock_code, o.stock_date, o.close_price,
        (SELECT oh.stock_date FROM ohlcv oh
        WHERE o.stock_code=oh.stock_code
        AND o.stock_date < oh.stock_date
        ORDER BY oh.stock_date asc
        LIMIT 1) AS b1_date,
        (SELECT oh.close_price FROM ohlcv oh
        WHERE o.stock_code=oh.stock_code
        AND o.stock_date < oh.stock_date
        ORDER BY oh.stock_date asc
        LIMIT 1) AS b1_price,
        (SELECT oh.stock_date FROM ohlcv oh
        WHERE o.stock_code=oh.stock_code
        AND o.stock_date+7 = oh.stock_date
        ORDER BY oh.stock_date asc) AS w1_date,
        (SELECT oh.close_price FROM ohlcv oh
        WHERE o.stock_code=oh.stock_code
        AND o.stock_date+7 = oh.stock_date
        ORDER BY oh.stock_date ASC ) AS w1_price
    FROM ohlcv AS o
    WHERE o.stock_code=005930;
    """

cursor.execute(sql)
samsung = cursor.fetchall()

df = pd.DataFrame(samsung)
df.columns = ["stock_code","stock_date", "close_price","b1_date","b1_price","w1_date","w1_price"]
df

,stock_code,stock_date,close_price,b1_date,b1_price,w1_date,w1_price
0,005930,2020-01-02,55200,2020-01-03,55500.0,2020-01-09,58600.0
1,005930,2020-01-03,55500,2020-01-06,55500.0,2020-01-10,59500.0
2,005930,2020-01-06,55500,2020-01-07,55800.0,2020-01-13,60000.0
3,005930,2020-01-07,55800,2020-01-08,56800.0,2020-01-14,60000.0
4,005930,2020-01-08,56800,2020-01-09,58600.0,2020-01-15,59000.0
5,005930,2020-01-09,58600,2020-01-10,59500.0,2020-01-16,60700.0
6,005930,2020-01-10,59500,2020-01-13,60000.0,2020-01-17,61300.0
7,005930,2020-01-13,60000,2020-01-14,60000.0,2020-01-20,62400.0
8,005930,2020-01-14,60000,2020-01-15,59000.0,2020-01-21,61400.0
9,005930,2020-01-15,59000,2020-01-16,60700.0,2020-01-22,62300.0
